In [1]:
#https://github.com/nhamlv-55/CFI_NNs/blob/master/MNIST_toy/Experiment-Feb28-xujie-Copy1.ipynb

from models import FeedforwardNeuralNetModel, TinyCNN, PatternClassifier
import regularizer_losts as rl
from torchvision import datasets, transforms
from torch import optim
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import copy
import utils as CFI_utils
from matplotlib import pyplot as plt
import numpy as np
from collections import defaultdict
#from frozendict import frozendict
from datetime import datetime
import seaborn as sn
import pandas as pd
%matplotlib inline

import seaborn as sns
colors = sns.color_palette("tab10")

#Logging stuffs
import logging
import sys
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

#configs
epochs = 10
batch_size = 1000
test_batch_size = 10000
stable_batch_size = 60000
use_cuda = False
lr = 0.01
log_interval = 100

#torch specific configs
torch.manual_seed(1)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}

test_kwargs = {'batch_size': test_batch_size}
stable_kwargs = {'batch_size': stable_batch_size}

if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)
    stable_kwargs.update(cuda_kwargs)
class Shift:
    def __init__(self, shift = 0):
        print("alive")
        self.shift = shift

    def __call__(self, arr):
        print("running")
        #print(arr)
        return arr
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}()"
    
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'landmarks': landmarks}


transform = transforms.Compose([
    transforms.ToTensor(),
   
    transforms.Normalize((0.1307,), (0.3081,))
   
  
])

def shift_and_roll(arr : torch.Tensor, x : int , y : int) -> torch.Tensor:
    return torch.roll(arr, shifts = (x,y), dims = (0,1)) #indexing
def shift(arr : torch.Tensor, x : int , y : int) -> torch.Tensor:

    pad = [0,0,0,0]
    x_start = 0
    x_end = 0
    y_start = 0
    y_end = 0
    if x >= 0:
        pad[0] = x
        x_start = 0
        x_end = 28
    else:
         pad[1] = abs(x)
         x_start = -28
         
         
        
         
    
    if y >= 0:
        pad[2] = y
        y_start = 0
        y_end = 28
    else:
        pad[3] = abs(y)
        y_start= -28
        print("ys is {}".format(y_start))
    

    padder = torch.nn.ZeroPad2d(tuple(pad))
  
    result = padder(arr)

    if y < 0:
        y_end = result.shape[0]
    if x < 0:
        x_end = result.shape[1]
    
    return result[y_start:y_end, x_start:x_end]

def noisify(arr : torch.Tensor , distribution : torch.distributions.Distribution) -> torch.Tensor: #randomly add noise
   
    #print(distribution.sample(arr.size()).shape\\)
    #print(torch.reshape(distribution.sample(arr.size()), (28,28)).shape)
   
    noise = torch.reshape(distribution.sample(arr.size()), (arr.shape))
    return arr + noise
transform = transforms.Compose([
    transforms.ToTensor(),
   
    transforms.Normalize((0.1307,), (0.3081,))
])


C:\Users\xxjtr\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def transform_dataset(dataset):
    '''
    DO A STATE CHANGE
    '''
    shift_x = 1
    shift_y = 0
    mu = 1
    sigma = 1
    do_shift = False
    do_noisfy = True
    modification_string = "base"
    if do_shift:
        modification_string += " -shift {} {} - ".format(shift_x, shift_y)
    if do_noisfy:
        modification_string += " -noise added using gaussian using mean {} and stddev {}- ".format(mu, sigma)
    for i in range(dataset.data.shape[0]):
        if do_shift:
            dataset.data[i,:,:] = shift(dataset.data[i,:,:],shift_x, shift_y)
        
        if do_noisfy:
            gaussian = torch.distributions.Normal(loc = mu, scale = sigma)# loc = mu, scale = stddev

            dataset.data[i,:,:] = noisify(dataset.data[i,:,:], gaussian)
    return modification_string

## Train/Load model


In [10]:
def check_gradient(grad, label, last_sorted_grads, plot = False):
    logging.info("CHECKING GRADIENT FOR LABEL {}".format(label))
    sum_abs_grad = np.sum(abs(grad[label]), axis = 0)
    
    current_sorted_grad = (-sum_abs_grad).argsort()
    
#     if len(last_sorted_grads[label]) > 0:
#         for k in [100, 200, 300, 400]:
#             prev_top_k = set(last_sorted_grads[label][-1][:k])
#             current_top_k = set(current_sorted_grad[:k])
#             intersect = prev_top_k.intersection(current_top_k)
#             logging.info('k = {}. How many top Gradients are stable since last epoch?: {}'.format(k, len(intersect)))
        
    for k in [0, 9, 99, 199]:    
        logging.debug('{}th biggest gradient = {}'.format(k, np.sort(-sum_abs_grad)[k]))
    if plot:
        fig = plt.figure(figsize=(30, 1))
        plt.bar(range(sum_abs_grad.shape[0]), sum_abs_grad)
        plt.show()
        print(sum_abs_grad.max(), sum_abs_grad.argmax(), sum_abs_grad.min())

    return current_sorted_grad



#init stuffs
LOAD = True
LOADPATH = 'FFN15-57-04'
LAST_N_EPOCHS = 10

dataset1 = datasets.MNIST('./data', train=True, download=False,
                          transform=transform)
dataset2 = datasets.MNIST('./data', train=False, download=False,
                          transform=transform)


#transform_dataset(dataset1)
#modification_string = transform_dataset(dataset2)
modification_string = ""
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = FeedforwardNeuralNetModel(28*28, 128, 10).to(device)
# model = TinyCNN().to(device)

if LOAD:
    model.load_state_dict(torch.load(LOADPATH))
else:

    epochs = 20
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=1, gamma = 0.7)
    last_sorted_grads = defaultdict(list)
    
    all_rows = []
    
    for epoch in tqdm(range(1, epochs + 1)):
        model.register_gradient()
        model.train()
        target_log  = None # need to record the label to match with the gradient later
        for data, target in train_loader:
            target_log = np.concatenate((target_log, target), axis = 0) if target_log is not None else target
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data.float())
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
        CFI_utils.test(model, device, test_loader)
#         scheduler.step()
        grad = CFI_utils.get_grad_each_label(model.gradient_log, 
                                      target_log = target_log, 
                                      layers = ['fc1', 'fc2', 'fc3', 'fc4'], 
                                      labels = range(10))
        torch.save(model.state_dict(), model.model_savename())
        
        row_data = []
        for label in range(10):
            r = []
            logging.info("After {} epoch:".format(epoch))
            last_sorted_grads[label].append(check_gradient(grad, label, last_sorted_grads))
            
            
            if epoch >= LAST_N_EPOCHS:
                for k in [100, 200, 300, 400]:
                    all_top_k = [set(sorted_grad[:k]) for sorted_grad in last_sorted_grads[label][-LAST_N_EPOCHS:]]
                    intersect = set.intersection(*all_top_k)
                    logging.info('k = {}. How many top Gradients are stable among all last {} epochs?: {}'.format(k, LAST_N_EPOCHS, len(intersect)))

                    

with torch.no_grad():
    total = 0
    correct = 0
    for data, target in dataset2:
        if int(torch.argmax(model.cpu()(data.cpu()), dim = 1)) == target:
#         if int(torch.argmax(model.cuda()(data.cuda()), dim = 1)) == target:
            correct += 1
        total+=1
    "accuracy {}".format( correct/total)
    modification_string += " -accuracy {}-".format( correct/total)
    print(modification_string)

 -accuracy 0.0974-


#### Compute all patterns in the training set, and put them into corresponding labels

In [5]:
class Patterns:
    def __init__(self, model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, labels, layers):
        self._model = model
        self.label2patterns = {}
        self.label2idx = {}
        self._labels = labels
        self._layers = layers
        self._dataloader = dataloader
        self._populate()
        
    def _populate(self):
        
        label2patterns = {}
        label2idx = {}
        for label in self._labels:
            patterns = []
            filter_ids = []
            
            for data, target in self._dataloader:
                
                flter = np.where(target == label)
                filter_ids.append(flter)
                data = data[flter]
                logging.debug(data.shape[0])
                pattern = self._model.get_pattern(data, layers, device, flatten = True)
                logging.debug(pattern.shape)
                patterns.append(pattern)

            patterns = np.squeeze(np.concatenate(patterns, axis = 0))
            filter_ids = np.squeeze(np.concatenate(filter_ids, axis = 0))
            label2patterns[label] = patterns
            label2idx[label] = filter_ids
            
            logging.info(patterns.shape)
        
        #freeze
        self.label2patterns = dict(label2patterns)
        self.label2idx = dict(label2idx)
        
    def apply_filter(self, f):
        pass
    
    def unique():
        pass
    
    def query_pattern():
        pass
    
    

layers = ['fc1', 'fc2', 'fc3', 'fc4']
labels = range(10)
K = 25
stable_loader = torch.utils.data.DataLoader(dataset1, **stable_kwargs)

all_patterns = Patterns(model = model,
                        dataloader = stable_loader,
                        labels = labels,
                        layers = layers)
all_test_patterns = Patterns(model = model,
                        dataloader = test_loader,
                        labels = labels,
                        layers = layers)



root - INFO - (5923, 458)
root - INFO - (6742, 458)
root - INFO - (5958, 458)
root - INFO - (6131, 458)
root - INFO - (5842, 458)
root - INFO - (5421, 458)
root - INFO - (5918, 458)
root - INFO - (6265, 458)
root - INFO - (5851, 458)
root - INFO - (5949, 458)
root - INFO - (980, 458)
root - INFO - (1135, 458)
root - INFO - (1032, 458)
root - INFO - (1010, 458)
root - INFO - (982, 458)
root - INFO - (892, 458)
root - INFO - (958, 458)
root - INFO - (1028, 458)
root - INFO - (974, 458)
root - INFO - (1009, 458)


In [6]:
#we can only study gradient if the model is trained
# FIXME: should save gradient into a pickle as well
if not LOAD:
    for K in [25, 50, 100]:
        print("K=", K)
        row = []
        row.append(str(K))
        for label in labels:
            #construct the stable gradients 
            all_top_k = [set(sorted_grad[:K]) for sorted_grad in last_sorted_grads[label][-LAST_N_EPOCHS:]]
            intersect = set.intersection(*all_top_k)
            stable_grad = np.array(sorted(list(intersect)))
            print("There are {} stable grad in top K".format(len(stable_grad)))
            print(stable_grad)

            logging.info(all_patterns[label].shape)
            print("LABEL:", label)
            print("how many unique paths in the full pattern?", np.unique(all_patterns[label], axis = 0).shape)
            print("how many unique paths in the filtered pattern?", np.unique(all_patterns[label][:, stable_grad ], axis = 0).shape)
            print("how many unique paths in the randomly filtered pattern?", 
                  np.unique(patterns[:, 
                                     np.random.choice(458, len(stable_grad), replace = False) ], axis = 0).shape)
            row.append("|".join([str(len(stable_grad)),
                                 str(np.unique(all_patterns[label], axis = 0).shape[0]),
                                 str(np.unique(all_patterns[label][:, stable_grad ], axis = 0).shape[0]),
                                 str(np.unique(all_patterns[label][:, np.random.choice(458, len(stable_grad), replace = False) ], axis = 0).shape[0])
                                ]))
        all_rows.append(",".join(row)+"\n")

    with open("gradient_exp_log_2.csv", "w") as f:
        f.write("K,"+",".join([str(l) for l in labels]) + "\n")
        f.writelines(all_rows)

K= 25
There are 0 stable grad in top K
[]


TypeError: 'Patterns' object is not subscriptable

In [12]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.bool_):
            return int(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    


all_stable_relus = []

all_alpha_patterns = {"model": LOADPATH}

write_log = True
if write_log:
    ReLU_exp_log = open("relu_exp_log{}.csv".format(datetime.now().strftime("%H-%M-%S")), "w")
    ReLU_exp_json = open("relu_exp_data{}.json".format(datetime.now().strftime("%H-%M-%S")), "w")
    ReLU_exp_log.write("Epsilon,Label,NumStableReLU,NumUniqueAP,Alpha Pattern Cover\n")



# for epsilon in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]:
epsilon_to_patterns = dict()
for epsilon in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]:
    alpha_patterns = {}
    label_to_most_common_pattern = dict()
    for label in all_patterns.label2patterns:
#     for label in [0,2,8]:
        patterns = all_patterns.label2patterns[label]
        print(patterns.shape)

        occuring_patterns = patterns.tolist() #make patterns into list of lists first dim is each example, second dim is the pattern
        most_common_pattern = max(occuring_patterns, key =  lambda x :occuring_patterns.count(x)) #argmax the count of each pattern

        #print("most common pattern: {}".format(most_common_pattern) )
        
        pattern_indices = list(filter( lambda x : most_common_pattern[x] , range(len(most_common_pattern))   ))#get true indices     
        #print("pattern_indices: {}".format(pattern_indices))
        label_to_most_common_pattern[label] = (pattern_indices,most_common_pattern)
        relu_sum = np.sum(patterns, axis = 0).squeeze()
        
        print("threshold: ", epsilon*patterns.shape[0], (1-epsilon)*patterns.shape[0])
        
#         print("relu_sum, layer-1 ", relu_sum[:256])
#         print("relu_sum, layer-2 ", relu_sum[256:384])
#         print("relu_sum, layer-3 ", relu_sum[384:448])
        print("relu_sum, prediction", relu_sum[448:458])
#         print("relu_sum", relu_sum[-10:])
        
        non_active_neurons = np.where(relu_sum<=epsilon*patterns.shape[0])
        active_neurons = np.where(relu_sum>=(1-epsilon)*patterns.shape[0])
        print("non active neurons: ", non_active_neurons)
        print("active neurons: ", active_neurons)

        stable_idx = np.concatenate([np.where(relu_sum<=epsilon*patterns.shape[0]), 
                                     np.where(relu_sum>=(1-epsilon)*patterns.shape[0])],
                                    axis = 1
                                    ).squeeze()
        neuro_idx = patterns.shape[1] - 10 + label
        if neuro_idx not in stable_idx:
            print(f"WARN: neuro_idx = {neuro_idx} for label {label} is not stable, let's include it anyway")
            stable_idx = np.append(stable_idx, neuro_idx)
        stable_idx = sorted(stable_idx) #sort the indices of the stable ReLUs. 
        unique_patterns, freq = np.unique(patterns[:, stable_idx ], axis = 0, return_counts=True)
        alpha_p = unique_patterns[np.argmax(freq)]
        print()
        print("Label is ", label)
        print("Stable ReLUs", stable_idx)
        print("how many unique paths in the filtered pattern?", unique_patterns.shape)
        print("their freq\n", freq, freq.shape)
#         print("most prominent pattern", np.argmax(freq), alpha_p)
#         print("alpha_p is ", alpha_p)


        assert(len(stable_idx) == alpha_p.shape[-1])
        assert(freq.shape[0]==unique_patterns.shape[0])
#         alpha_patterns[label] = (stable_idx, tuple(alpha_p))
#         alpha_patterns[label] = {"stable_idx": stable_idx,
#                                 "alpha_pattern": alpha_p,
#                                 "alpha_pattern_coverage": freq.max()/freq.sum(),
#                                 "pattern_frequency": freq}
        alpha_patterns[label] = {"stable_idx": stable_idx,
                                 "active_neurons": active_neurons[0],
                                 "non_active_neurons": non_active_neurons[0],
                                "alpha_pattern": alpha_p,
                                "alpha_pattern_coverage": freq.max()/freq.sum(),
                                "pattern_frequency": freq}
    
#         print("pattern frequency:", freq)
        print("primary pattern coverage: ", freq.max()/freq.sum(),)

        if write_log:
            ReLU_exp_log.write("{},{},{},{},{}\n".format(epsilon, label, len(stable_idx), unique_patterns.shape[0], freq.max()/freq.sum()))
    all_alpha_patterns[epsilon] = alpha_patterns
    epsilon_to_patterns[epsilon] = label_to_most_common_pattern
json.dump(epsilon_to_patterns, open("most_common_patterns{}.json".format(datetime.now().strftime("%H-%M-%S")), "w")) 
    
if write_log:
    ReLU_exp_log.close()
    json.dump(all_alpha_patterns, fp = ReLU_exp_json, indent=2, cls=NpEncoder)
    ReLU_exp_json.close()

(5923, 458)
threshold:  0.5923 5922.4077
relu_sum, prediction [   0    0    0    0    0    0    0    0 5923    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,

threshold:  0.5958 5957.4042
relu_sum, prediction [   0    0    0    0    0    0    0    0 5958    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 264, 265, 266, 269, 270,
       272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289, 300, 302,
       303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325, 326, 327,
       331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367, 371, 376,
       378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398, 399, 400,
       403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431, 432, 434,
       436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452, 453, 454,
       455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  

threshold:  0.5842 5841.4158
relu_sum, prediction [   0    0    0    0    0    0    0    0 5842    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  

threshold:  0.5918 5917.4082
relu_sum, prediction [   0    0    0    0    0    0    0    0 5918    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  

threshold:  0.5851000000000001 5850.4149
relu_sum, prediction [   0    0    0    0    0    0    0    0 5851    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,

...

threshold:  296.15000000000003 5626.849999999999
relu_sum, prediction [   0    0    0    0    0    0    0    0 5923    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
   

threshold:  297.90000000000003 5660.099999999999
relu_sum, prediction [   0    0    0    0    0    0    0    0 5958    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
   

threshold:  292.1 5549.9
relu_sum, prediction [   0    0    0    0    0    0    0    0 5842    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59, 

threshold:  295.90000000000003 5622.099999999999
relu_sum, prediction [   0    0    0    0    0    0    0    0 5918    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
   

threshold:  292.55 5558.45
relu_sum, prediction [   0    0    0    0    0    0    0    0 5851    0]
non active neurons:  (array([ 11,  13,  16,  19,  70,  87, 118, 181, 257, 261, 264, 265, 266,
       269, 270, 272, 275, 276, 277, 279, 280, 282, 285, 286, 288, 289,
       300, 302, 303, 304, 308, 310, 311, 312, 314, 317, 321, 324, 325,
       326, 327, 331, 335, 340, 341, 346, 350, 351, 352, 364, 366, 367,
       371, 376, 378, 380, 381, 382, 383, 385, 389, 391, 393, 395, 398,
       399, 400, 403, 405, 407, 409, 412, 417, 418, 426, 429, 430, 431,
       432, 434, 436, 440, 441, 444, 445, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 457], dtype=int64),)
active neurons:  (array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  14,
        15,  17,  18,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59

## Apply delta debugging

In [60]:
import matplotlib.pyplot as plt
from custom_transform import *
from models import MNISTNet
from transforms import TRANSFORMS
from mask import *
import imageio
from utils import generate_gif
import os

# For delta debugging
img_eraser = CustomizeMask(value=0)

# Parameters
THRESHOLD = 0.6
ROOT_FOLDER = "./"
TMP_IMG_FOLDER = os.path.join(ROOT_FOLDER, "mnistact_gif_img")

RESULT_FOLDER_NAME = "result_samples_mnistact"
GIF_RESULT = ROOT_FOLDER + RESULT_FOLDER_NAME + "/{}/gif_results/img_{}.gif"
IMG_RESULT = ROOT_FOLDER + RESULT_FOLDER_NAME + "/{}/img_results/img_{}.png"

# Create folders for save all the results, only run once
create_folders = False
if create_folders:
#     os.mkdir(ROOT_FOLDER + RESULT_FOLDER_NAME)
    for i in range(10):
        os.mkdir(ROOT_FOLDER+RESULT_FOLDER_NAME+"/{}/".format(i))
        os.mkdir(ROOT_FOLDER+RESULT_FOLDER_NAME+"/{}/{}".format(i,'gif_results'))
        os.mkdir(ROOT_FOLDER+RESULT_FOLDER_NAME+"/{}/{}".format(i,'img_results'))
# MODEL_PATH = "./cifar10_net_v2.pth"

epsilon_for_pattern = 0.05

# We have loaded the dataset already at the beginning

def check_activation(data, label, device, active_neurons_idx, non_active_neurons_idx):
    
    img_pattern = model.get_pattern(data, layers, device, flatten = True)
    img_non_active_neurons = img_pattern[:, non_active_neurons_idx]
    img_active_neurons = img_pattern[:, active_neurons_idx]
#     print("NONNNN", img_pattern.shape, img_non_active_neurons, img_non_active_neurons.sum(), len(non_active_neurons_idx), non_active_neurons_idx)
#     print("aaaaa", img_active_neurons.sum(), len(active_neurons_idx), active_neurons_idx)

    
    return img_non_active_neurons.sum() == len(non_active_neurons_idx) and img_active_neurons.sum() == len(active_neurons_idx)


def delta_debugging_general_new(img_eraser, model, threshold, img, label):

    model.eval()
    
#     active_neurons_idx = all_alpha_patterns[epsilon_for_pattern][label]["active_neurons"]
#     non_active_neurons_idx = all_alpha_patterns[epsilon_for_pattern][label]["non_active_neurons"]
    active_neurons_idx = all_alpha_patterns[epsilon_for_pattern][label]["active_neurons"][0]
    non_active_neurons_idx = all_alpha_patterns[epsilon_for_pattern][label]["non_active_neurons"][0]
#     stable_idx = all_alpha_patterns[epsilon_for_pattern][label]["stable_idx"]
#     stable_idx = np.concatenate([active_neurons_idx, 
#                                      non_active_neurons_idx],
#                                     axis = 1
#                                     ).squeeze()
#     print([active_neurons_idx, non_active_neurons_idx], len(stable_idx), stable_idx)
    
    c, h, w = img.shape

    current_unmasked_pixels = torch.ones((c, h, w))
    
    keep_going = True
    smaller_chunk = True

    current_img = img.clone()
    current_pred = None
    current_pred_prob = 1

    prev_img = img.clone()
    prev_pred = None
    prev_pred_prob = 1
    

    img_counter = 0
    img_names = []
    smaller_group_number = None  
#     img_folder = "./gif_img"
    if not os.path.exists(TMP_IMG_FOLDER):
        os.mkdir(TMP_IMG_FOLDER)
    if smaller_chunk:
        smaller_group_number = 2
#         for _ in range(1):

        while keep_going:
            prev_img, prev_pred, prev_pred_prob = current_img, current_pred, current_pred_prob
            
            # save figures for gif
            
            figure = plt.figure()
            plt.title("Pred prob: {}".format(prev_pred_prob))
            plt.imshow(prev_img.reshape(28, 28), cmap="gray")
            img_name = '{}.png'.format(img_counter)
            plt.savefig(os.path.join(TMP_IMG_FOLDER, img_name))
            plt.close(figure)
            img_names.append(img_name)
            img_counter += 1

            pixel_left = int(torch.sum(current_unmasked_pixels).item())
            if smaller_group_number > pixel_left:
                smaller_group_number = pixel_left
            
            try:
                all_new_masks = split_in_smaller_groups(current_unmasked_pixels, smaller_group_number)
            except:
                keep_going = False
                continue
                
            best_group_num = None
            best_pred_prob = 0
            for m_idx in range(smaller_group_number):
                new_unmask = all_new_masks[m_idx]
                new_img = img_eraser(img, new_unmask)
                new_extend_img = new_img[None, :]
                with torch.no_grad():
                    output = model(new_extend_img)
                    # TODO: check image pattern
                    same_activation = check_activation(new_extend_img, label, device, active_neurons_idx, non_active_neurons_idx)
                    print(same_activation)
                new_pred_prob = torch.max(torch.exp(output)).item()
                new_pred = torch.argmax(output).item()
                if new_pred_prob > threshold and new_pred == label and new_pred_prob > best_pred_prob and same_activation:
                    current_unmasked_pixels = new_unmask
                    current_img, current_pred, current_pred_prob = new_img, new_pred, new_pred_prob
                    best_group_num = m_idx

            if best_group_num is None:
                if smaller_group_number == pixel_left:
                    keep_going = False
                else:
                    smaller_group_number += 1
                
    print("in the end:", smaller_group_number)
    generate_gif(GIF_RESULT.format(label, img_idx), TMP_IMG_FOLDER, img_names, True)
    return prev_img , prev_pred, prev_pred_prob



In [ ]:
for img_idx in range(0, 1):
    (img, label) = torch.utils.data.Subset(dataset2, [img_idx])[0]
#     if label != 8:
#         continue
#     plt.figure()
#     plt.imshow(img.reshape(28, 28), cmap="gray")

    pred_img, pred_class, pred_prob = delta_debugging_general_new(img_eraser, model, 0.6, img, label)
# #     plt.figure()
# #     plt.imshow(pred_img.reshape(28, 28), cmap="gray")

#     figure = plt.figure()
#     f, axarr = plt.subplots(1,2)
#     axarr[0].imshow(img.reshape(28, 28), cmap="gray")
#     axarr[1].imshow(pred_img.reshape(28, 28), cmap="gray")
#     plt.title("Final pred prob: {}".format(pred_prob))
#     plt.savefig("./result_samples/{}/img_results/img_{}.png".format(label, img_idx))
#     plt.close(figure)

    
    


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

...

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals